This file generates the social networks under different node degree coefficients. The main purpose is to obtain the network matrix to see how resources are shared through different types of social ties.

In [5]:
import numpy as np
import scipy.sparse as sp
import pandas as pd

import sys
sys.path.append('..')
import src

We first examine how the quantity of social ties in a community affects community resilience. To explore this aspect, we multiply the node degrees by different scaling factors,
namely 0.2, 0.4, ..., and 2.0, and round them to the nearest integer in the social network generation process, while keeping all other steps unchanged.

In [6]:
PARAMS = {
    'community': 'laurelhurst',
    'experiment_name': 'node_degree',
    # resource types should be the same as the column names in the resource_preference_pmf and resource_inventory_pmf
    'resource_types': ['water', 'food', 'meds', 'transp', 'comm',
                       'faid', 'warm', 'sani', 'power', 'shelter'], 
    'tau': 5, # isolation days
    'num_scenarios': 100,
    'n': 0.78773808, # degree distribution parameter
    'p': 0.02559534, # degree distribution parameter
    'social_tie': {
        'types': ['strong', 'weak'],
        'pmf':  [0.41296, 0.58704]
    },
    'distance_decay_alpha': -1.35, # distance decay parameter
    'distances_mat': np.load('../temp/laurelhurst_distance_matrix.npy'), # distance matrix
    'sharing_preference_pmf': pd.read_csv('../temp/laurelhurst_share_pref_PMF.csv'), # sharing preference matrix
    'sharing_priority': [3, 2, 1], # sharing priorities to [strong tie, weak tie, stranger]
    'resource_inventory_pmf': pd.read_csv('../temp/laurelhurst_resource_distr_PMF.csv') # resource inventory distribution
}

In [7]:
def p2p_single_resource_sharing(n:float, # negative binomial distribution parameter
                                p:float, # negative binomial distribution parameter
                                degree_ratio:float, # degree ratio
                                distance_decay_alpha:float, # distance decay parameter
                                distance_mat:np.matrix, # distance matrix
                                social_tie_pmf:list,
                                social_tie_types:list = ['strong', 'weak'],
                                seed:int=0):
    """Step 1: Community-based social network construction"""
    # create a community
    community = src.Community()
    num_nodes = distance_mat.shape[0]
    community.add_nodes_from(range(num_nodes))

    # generate social ties
    np.random.seed(seed)
    degree_list = np.random.negative_binomial(n, p, size=num_nodes)
    degree_list = degree_list * degree_ratio
    degree_list = degree_list.astype(int)
    community.generate_social_ties(degrees=degree_list, 
                                   distance_matrix=distance_mat, 
                                   distance_decay_alpha=distance_decay_alpha, 
                                   seed=seed+1)
    community.split_social_ties(types=social_tie_types, 
                                pmf=social_tie_pmf, 
                                seed=seed+2)
    
    strong_tie_matrix = community.get_social_tie_matrix(type='strong')
    weak_tie_matrix = community.get_social_tie_matrix(type='weak')

    return strong_tie_matrix, weak_tie_matrix

In [8]:
# main script for running the status quo model
for degree_ratio in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:

    for sce in range(PARAMS['num_scenarios']):
        
        # directory name for saving the results
        dir = PARAMS['community'] + '/' + PARAMS['experiment_name'] + '/tau_' + str(PARAMS['tau']) # directory name
        # Create a dataframe to store the results
        inventory_df = pd.DataFrame(columns=PARAMS['resource_types'])
        resource_df = pd.DataFrame(columns=PARAMS['resource_types'])
        
        # run the model for each resource type
        strong_tie_matrix, weak_tie_matrix = p2p_single_resource_sharing(
                                                                    n=PARAMS['n'],
                                                                    p=PARAMS['p'],
                                                                    degree_ratio=degree_ratio,
                                                                    social_tie_types = PARAMS['social_tie']['types'],
                                                                    social_tie_pmf = PARAMS['social_tie']['pmf'],
                                                                    distance_decay_alpha = PARAMS['distance_decay_alpha'],
                                                                    distance_mat = PARAMS['distances_mat'],
                                                                    seed=sce)
        # save sparse matrix
        sp.save_npz(dir + '/strong_tie_matrix_degree_{}_sce_{}.npz'.format(degree_ratio, sce), sp.csr_matrix(strong_tie_matrix))
        sp.save_npz(dir + '/weak_tie_matrix_degree_{}_sce_{}.npz'.format(degree_ratio, sce), sp.csr_matrix(weak_tie_matrix))